# Third Notebook for Capstone Project (by Glenn Tiffert)

## Import dependencies

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

!conda install -c conda-forge geopy --yes 

import folium
import json
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize

failed

CondaError: KeyboardInterrupt



## Get Wikipedia page and parse with BeautifulSoup

In [2]:
url=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(url.text, 'lxml')

In [3]:
table = soup.find('table', class_='wikitable sortable')
table_body = table.find('tbody')
print(table_body)

<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North_York" tit

In [4]:
data = []
columns = table_body.tr.text.split()
rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) # Get rid of empty values

## Create dataframe, drop rows with unassigned boroughs, replace unassigned neighbourhoods with borough name, re-index

In [5]:
df = pd.DataFrame(data, columns=columns)
df = df.drop(0)
df = df.reset_index(drop=True)
df =df.loc[df['Borough'] != 'Not assigned']
df.Neighbourhood[df.Neighbourhood == 'Not assigned'] = df.Borough
df = df.reset_index(drop=True)

## Combine neighborhoods that share a postcode, display final dataframe

In [6]:
df_new=(df.astype(str).groupby('Postcode')['Borough','Neighbourhood']
    .agg({'Borough':'first','Neighbourhood':lambda x: ', '.join(x)}).reset_index())
df_new

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


## Get shape of dataframe

In [7]:
df_new.shape

(103, 3)

## Fetch latitude and longitude coordinates for neighborhoods using geocoder package

In [8]:

# This cell is commented out because geocoder did not work after repeated tries, so I fell back to Plan B: 
# using the supplied csv file instead.


"""
import geocoder # import geocoder

lat_cords = []
long_cords = []

for postal_code in range(len(df_new['Postcode'])):
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng

    lat_cords.append(lat_lng_coords[0])
    lat_cords.append(lat_lng_coords[1])
    
print (lat_cords, long_cords)
"""

"\nimport geocoder # import geocoder\n\nlat_cords = []\nlong_cords = []\n\nfor postal_code in range(len(df_new['Postcode'])):\n    # initialize your variable to None\n    lat_lng_coords = None\n\n    # loop until you get the coordinates\n    while(lat_lng_coords is None):\n      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))\n      lat_lng_coords = g.latlng\n\n    lat_cords.append(lat_lng_coords[0])\n    lat_cords.append(lat_lng_coords[1])\n    \nprint (lat_cords, long_cords)\n"

## Geocoder did not work. Use supplied .csv file instead

In [9]:
url = 'https://cocl.us/Geospatial_data'
coord = pd.DataFrame(pd.read_csv(url))
coord = coord.rename (columns={'Postal Code':'Postcode'}) #rename column to match first dataframe for indexing

## Merge the two dataframes

In [10]:
final = pd.merge(df_new,coord, on='Postcode')
final

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


## Map the Post Codes

In [11]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(final['Latitude'], final['Longitude'], final['Borough'], final['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Code Foursquare credentials

In [12]:
CLIENT_ID = 'EL3LAJ30HQ0K5WUPMCMHIIR4VJLQINFK5FTWRD0TY4HX3QKP' # your Foursquare ID
CLIENT_SECRET = 'ON3B4BWPMPZCD1HYCQQBAFPEH2CRMQ1QFHOVHFTB3W2GR2JY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EL3LAJ30HQ0K5WUPMCMHIIR4VJLQINFK5FTWRD0TY4HX3QKP
CLIENT_SECRET:ON3B4BWPMPZCD1HYCQQBAFPEH2CRMQ1QFHOVHFTB3W2GR2JY


## Define a function to get venues near post codes coordinates

In [13]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

def getNearbyVenues(names, latitudes, longitudes, radius=750):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
toronto_venues = getNearbyVenues(names=final['Neighbourhood'],
                                   latitudes=final['Latitude'],
                                   longitudes=final['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

## Show number of venues found per neighbourhood

In [15]:
t_venues = toronto_venues[['Neighbourhood','Venue']]
t_venues.groupby('Neighbourhood').count()

# toronto_venues.groupby('Neighbourhood').count()

,Venue
Neighbourhood,
"Adelaide, King, Richmond",100
Agincourt,16
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",19
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",13
"Alderwood, Long Branch",14
"Bathurst Manor, Downsview North, Wilson Heights",21
Bayview Village,11
"Bedford Park, Lawrence Manor East",34
Berczy Park,100


In [16]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 323 uniques categories.


## Analyze neighbourhoods

In [17]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

## Display venues by type, grouped by neighbourhood

In [18]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waste Facility,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.010000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.00,0.000000,0.000000,0.000000
1,Agincourt,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000
4,"Alderwood, Long Branch",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000
5,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,...,0.000000,0.047619,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000
6,Bayview Village,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000
7,"Bedford Park, Lawrence Manor East",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,...,0.000000,0.029412,0.000000,0.000000,0.000000,0.00,0.00,0.029412,0.000000,0.000000
8,Berczy Park,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000
9,"Birch Cliff, Cliffside West",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000


## List top 5 venues by neighbourhood

In [19]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
              venue  freq
0       Coffee Shop  0.07
1              Café  0.06
2        Steakhouse  0.04
3  Sushi Restaurant  0.04
4   Thai Restaurant  0.03


----Agincourt----
                      venue  freq
0                 Pool Hall  0.06
1            Discount Store  0.06
2           Motorcycle Shop  0.06
3  Mediterranean Restaurant  0.06
4               Supermarket  0.06


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                  venue  freq
0              Pharmacy  0.11
1  Fast Food Restaurant  0.11
2    Chinese Restaurant  0.11
3           Pizza Place  0.11
4  Caribbean Restaurant  0.05


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0         Grocery Store  0.23
1           Pizza Place  0.15
2              Pharmacy  0.08
3  Fast Food Restaurant  0.08
4          Liquor Store  0.08


----Alderwood, Long Branch----
       

                   venue  freq
0          Shopping Mall  0.14
1                   Bank  0.14
2            Coffee Shop  0.14
3  Vietnamese Restaurant  0.14
4            Pizza Place  0.14


----Downsview, North Park, Upwood Park----
                        venue  freq
0                Home Service  0.50
1  Construction & Landscaping  0.25
2                      Bakery  0.25
3           Accessories Store  0.00
4                 Music Venue  0.00


----East Birchmount Park, Ionview, Kennedy Park----
                 venue  freq
0          Coffee Shop  0.20
1    Convenience Store  0.13
2       Discount Store  0.07
3                 Bank  0.07
4  Rental Car Location  0.07


----East Toronto----
              venue  freq
0       Pizza Place  0.10
1              Café  0.10
2       Coffee Shop  0.10
3         Gastropub  0.05
4  Greek Restaurant  0.05


----Emery, Humberlea----
                           venue  freq
0                   Home Service  0.17
1              Convenience Store  0.17
2 

             venue  freq
0      Coffee Shop  0.07
1   Clothing Store  0.06
2   Cosmetics Shop  0.04
3  Bubble Tea Shop  0.03
4        Gastropub  0.03


----Scarborough Village----
                  venue  freq
0  Fast Food Restaurant  0.33
1           Coffee Shop  0.11
2     Convenience Store  0.11
3            Restaurant  0.11
4           Pizza Place  0.11


----Silver Hills, York Mills----
               venue  freq
0               Pool   1.0
1  Accessories Store   0.0
2       Music School   0.0
3       Noodle House   0.0
4          Nightclub   0.0


----St. James Town----
         venue  freq
0  Coffee Shop  0.07
1         Café  0.06
2        Hotel  0.05
3   Restaurant  0.05
4       Bakery  0.04


----Stn A PO Boxes 25 The Esplanade----
                 venue  freq
0          Coffee Shop  0.11
1                 Café  0.06
2           Restaurant  0.05
3                Hotel  0.04
4  Japanese Restaurant  0.04


----Studio District----
            venue  freq
0     Coffee Shop  0.06
1 

## Find most common venues across neighbourhoods

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Sushi Restaurant,Steakhouse,Bar,Thai Restaurant,Theater,Hotel,American Restaurant,Pizza Place
1,Agincourt,Mediterranean Restaurant,Restaurant,Shopping Mall,Supermarket,Sandwich Place,Skating Rink,American Restaurant,Discount Store,Shanghai Restaurant,Pool Hall
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Fast Food Restaurant,Pizza Place,Chinese Restaurant,Pharmacy,Gym,BBQ Joint,Hobby Shop,Shop & Service,Coffee Shop,Noodle House
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Hardware Store,Sandwich Place,Liquor Store,Pharmacy,Beer Store,Fried Chicken Joint,Coffee Shop,Fast Food Restaurant
4,"Alderwood, Long Branch",Pizza Place,Park,Pool,Dance Studio,Coffee Shop,Gym,Pub,Convenience Store,Gas Station,Pharmacy


## Cluster neighbourhoods using k-Means

In [22]:
# set number of clusters
kclusters = 8

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 1, 1, 1, 0, 0, 0, 0], dtype=int32)

In [23]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = final

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged['Cluster Labels']=toronto_merged['Cluster Labels'].fillna(0).astype(np.int64)

toronto_merged.head(25) # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0,Fast Food Restaurant,Coffee Shop,Martial Arts Dojo,Spa,Paper / Office Supplies Store,Yoga Studio,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0,Breakfast Spot,Italian Restaurant,Bar,Burger Joint,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1,Pizza Place,Fast Food Restaurant,Grocery Store,Beer Store,Breakfast Spot,Fried Chicken Joint,Greek Restaurant,Sports Bar,Coffee Shop,Comic Shop
3,M1G,Scarborough,Woburn,43.770992,-79.216917,4,Coffee Shop,Park,Business Service,Convenience Store,Eastern European Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Bakery,Pharmacy,Coffee Shop,Indian Restaurant,Yoga Studio,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Music Store,Athletics & Sports
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,1,Fast Food Restaurant,Coffee Shop,Pizza Place,Pharmacy,Restaurant,Sandwich Place,Convenience Store,Yoga Studio,Donut Shop,Dim Sum Restaurant
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,0,Coffee Shop,Convenience Store,Metro Station,Light Rail Station,Bus Station,Department Store,Sandwich Place,Grocery Store,Bank,Rental Car Location
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,0,Bus Line,Coffee Shop,Bakery,Diner,Fast Food Restaurant,Convenience Store,Bus Station,Intersection,Soccer Field,Park
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476,1,Chinese Restaurant,Wings Joint,Furniture / Home Store,Pizza Place,Burger Joint,Yoga Studio,Discount Store,Dive Bar,Dog Run,Doner Restaurant
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,0,College Stadium,Construction & Landscaping,Bank,Skating Rink,Discount Store,Thai Restaurant,Diner,General Entertainment,Café,Convenience Store


## Map clusters

In [24]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Map neighbourhoods with a Thai restaurant

In [25]:
thai = pd.merge(toronto_onehot[['Neighbourhood','Thai Restaurant']],final, on='Neighbourhood')
indexNames = thai[ thai['Thai Restaurant'] == 0 ].index
thai.drop(indexNames, inplace=True)

In [26]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(thai['Latitude'], thai['Longitude'], thai['Neighbourhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Finding: Thai restaurants in Toronto are distributed unevenly, and favor downtown neighbourhoods greatly.